<div>
    <img style="float:right;" src="images/smi-logo.png"/>
    <div style="float:left;color:#58288C;"><h1>Datenanalyse und Datenmanagement</h1></div>
</div>

---
# Notebook IV: Data Analytics
In diesem Notebook soll anhand des bekannten Kreditrisiko-Beispiels mit Machine Learning Modellen eine Vorhersage über Kreditwürdigkeit eines Antrages gemacht werden.
Hierzu werden zunächst die Merkmale entsprechend aufbereitet, damit die Machine Learning Modelle damit rechnen können.

## Inhaltsverzeichnis

[1. Einstieg: Research Approach](#kapitel1)  
[2. Datenaufbereitung](#kapitel2)  
[3. Modellbildung](#kapitel3)  
[4. Modellevaluation](#kapitel4)  
[5. Ausblick: Unsupervised K-Means Clustering](#kapitel5)  

---

## 1. Einstieg: Research Approach <a id="kapitel1"/>

- **Business Problem**: Nach Eingang eines Kreditantrags muss über die Vergabe und den angebotenen Zinssatz entschieden werden.  
Diese Entscheidung hängt vom angenommenen Ausfallrisiko des Kredits ab.
- **Research Problem**: Das Modell soll jeden Antrag klassifizieren: Risiko vs. kein-Risiko.  
Die Entscheidung über Vergabe und Zinssatz wird basierend auf dieser Information vom jew. Sachbearbeiter nach separat verfassten Richtlinien getroffen.
- **Trainingsdaten**: Vergangene Kreditanträge und Ausfall j/n

## 2. Datenaufbereitung <a id="kapitel2"/>

### 2.1. Daten einlesen
Auf bekannte Weise werden zunächst Pakete importiert und die Daten aus der Datenbank abgefragt. Über die Bekannten hinaus verwenden wir in diesem Notebook das Paket [scikit-learn](https://scikit-learn.org/stable/).

In [ ]:
import pandas as pd
import sklearn.tree
import sklearn.linear_model
import sklearn.model_selection
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext sql

# Visualisierung-Stil für Diagramme mit Seaview setzen
sns.set(font_scale = 1.1,
       palette     = "pastel", 
       style       = "whitegrid")

Falls die SQLite-Datenbank noch nicht entpackt ist, das jetzt tun:

In [ ]:
![ ! -f data/smi-data.db ] && unzip -o -d data data/smi-data.zip

In [ ]:
# Verbindung zum Datenbankserver herstellen
%sql sqlite:///data/smi-data.db

# SQL-Abfrage durchführen und Ergebnis in Variable result speichern        
result = %sql SELECT * FROM credit_ger

# Aus Result ein DataFrame machen
df = result.DataFrame()

Nun setzen wir den Primärschlüssel als Index des DataFrames und benennen die Merkmale griffiger (keiner Leerzeichen, nur Kleinschreibung, ...).

In [ ]:
df = df.set_index(["id"])
df = df.rename({
    "Age": "age",
    "Sex": "sex",
    "Job": "job",
    "Housing": "housing",
    "Saving_accounts": "savings", 
    "Checking_account": "cash",
    "Credit_amount": "amount",
    "Duration": "duration",
    "Purpose": "purpose",
    "Risk": "risk"
}, axis="columns")
df.purpose = df.purpose.str.slice(0,8)   # Text in der Purpose-Spalte auf max. 8 Zeichen kürzen für bessere Lesbarkeit
df.head(5)

### 2.2. Feature Engineering & Encoding

Worum geht es?
>Feature Engineering ist der Prozess, Merkmale für die Algorithmen möglichst gut zugänglich zu machen. Dabei fließt in der Regel Domänenwissen des Modellierers in den Datensatz ein.

In unserem Beispiel begnügen wir uns mit dem Minimum: Die gewählten Machine Learning Verfahren sollen technisch mit dem Datensatz umgehen können. 
So finden sich etwa mehrere Textmerkmale im Datensatz, mit denen nicht direkt gerechnet werden kann. Diese sind entsprechend umzukodieren.

Ziel ist die Zusammenstellung eines Trainingsdatensatzes bestehend aus 
- einem Dataframe X mit den Merkmalsausprägungen und 
- einem Dataframe y mit den richtigen Klassifikationsergebnissen (Kreditwürdig ja/nein).

### 2.2.1. Beispiel

In [ ]:
# Die One-Hot-Codierung ist eine einfache Kodierform, die ein Merkmal mit mehreren Textausprägungen in mehrere 0/1-Merkmale umwandelt:

beispiel = df["housing"]
one_hot_encoded = pd.get_dummies(beispiel)
pd.concat([beispiel, one_hot_encoded], axis=1).head(10)

### 2.2.2. Anwendung One-Hot Encoding

In [ ]:
# Was ist zu tun?

binaere_merkmale = ["sex", "risk"]                   # Umkodieren in 0/1
nominale_merkmale = ["housing", "purpose"]           # One-Hot-Encoding
ordinale_merkmale = ["savings", "cash", "job"]       # Umkodieren in Wertebereich 0..3
metrische_merkmale = ["age", "amount", "duration"]   # keine Umkodierung erforderlich

In [ ]:
# Zunächst erzeugen wir DataFrames für die aufbereiteten Trainingsdaten (X) 
# und Labels (y)

X = pd.DataFrame()
y = pd.DataFrame()

# 0/1-Kodierung der binären Merkmale

y["risk"] = pd.get_dummies(df.risk).bad   # das "bad" Dummymerkmal übernehmen wir als Zielgröße "risk"
X["male"] = pd.get_dummies(df.sex).male   # das "male" Dummymerkmal übernehmen wir als erstes Merkmal in das Merkmalsdataframe X

# One-Hot-Kodierung der nominalen und ordinalen Merkmale

kodierte_merkmale = pd.get_dummies(df[nominale_merkmale + ordinale_merkmale])
X = pd.concat([X, kodierte_merkmale], axis=1)   # concat fügt zwei Datasets zusammen, hier wird X das Dataset "kodierte_merkmale" hinzugefügt

# Metrische Merkmale anfügen

X = pd.concat([X,df[metrische_merkmale]], axis=1)

In [ ]:
X.head()

## 3. Modellbildung <a id="kapitel3"/>
Im ersten Schritt legen wir ein paar Daten beiseite, um daran unser Modell später zu testen. Man spricht hier von der Aufteilung in Trainings- und Testdaten.  

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=0)   # 20% der Daten werden als Testdaten beiseite gelegt

### 3.1. Aufbau eines Entscheidungsbaums (Decision Tree)
Ziel soll nun sein, das Kreditrisiko aus den anderen Merkmalen mittels eines Entscheidungsbaums vorherzusagen.  

In [ ]:
tree = sklearn.tree.DecisionTreeClassifier(min_samples_leaf=20)
tree = tree.fit(X_train, y_train)    # zur Erinnerung: X enthält unsere aufbereiteten Fallmerkmale, X_train nur die Fälle, die wir als Trainingsdaten verwenden wollen

Visualisieren wir nun den Baum um die Art der Regeln zu begutachten. Jeder Knoten enthält dabei folgende Werte:
- Split-Kriterium
- Gini-Koeffizient (ignorieren wir im Rahmen dieser Veranstaltung)
- Anteil der Fälle in diesem Knoten
- Anteil der Fälle mit risk 0 und risk 1
- Entscheidung für welche Klasse (y0, y1)

Der linke Pfeil bedeutet "True", der rechte Pfeil "False" bezogen auf das Knotenkriterium. Die Färbung des Knotens zeigt, wie das Zielkriterium (risk) hier ausgeprägt ist.

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(35,15))
plt.style.use('default')  # Bug in scikit-learn: Wenn Seaborn-Style gesetzt, wird der Tree nicht korrekt dargestellt, daher erst zurücksetzen
t = sklearn.tree.plot_tree(tree, ax=ax, label="root", precision=2, rounded=True, feature_names=X.columns, class_names=["no-risk","risk"], fontsize=12, proportion=True, filled=True)
plt.show()

## 4. Modellevaluation <a id="kapitel"/>

Als vermutlich einfachste Kennzahl können wir uns die durchschnittliche Treffergenauigkeit (% der Fälle die korrekt klassifiziert wurden) angeben lassen:

In [ ]:
print("Average Precision DecisionTree bei Trainingsdaten", tree.score(X_train, y_train))
print("Average Precision DecisionTree bei Testdaten", tree.score(X_test, y_test))